In [ ]:
import numpy as np
import tifffile as tiff

import matplotlib.pyplot as plt
from pathlib import Path


source = Path(r'C:/Users/miche/Documents/MATLAB/LENS Projects/Vignetting/CIDRE')
outdir = Path('C:/Users/miche/Desktop/HBP/Tools/mic/models/lsfm_lavision12x_dskwd')


# emission wavelength [nm]
sample = 'I48'
wl = 638
view = 'l'

# Broca's area slice ID
slc = 36


**View flat-field model**

In [ ]:
import scipy.io
import matplotlib.ticker as ticker


v = scipy.io.loadmat(source / f'v_dspim_S{slc}.mat')['v']
z = scipy.io.loadmat(source / f'z_dspim_S{slc}.mat')['z']

orig_shape = v.shape
print(f"Original flat-field model shape: {orig_shape}")

if orig_shape[0] > orig_shape[1]:
    v = np.swapaxes(v, 0, 1)
    z = np.swapaxes(z, 0, 1)
    

fig, ax = plt.subplots(figsize=(12, 4))
ax.imshow(v)
ax.set_aspect('auto')
ax.set_yticks([])
ax.minorticks_on()
ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))  # Major ticks every 1000
ax.xaxis.set_minor_locator(ticker.MultipleLocator(500))   # Minor ticks every 500
plt.show()


**Manual crop by visual inspection**

In [ ]:
crop_s = 750
crop_e = 8750

v = v[:, crop_s:crop_e]
z = z[:, crop_s:crop_e]

fig, ax = plt.subplots(figsize=(12, 4))
ax.imshow(v)
ax.set_aspect('auto')
ax.set_yticks([])
ax.minorticks_on()
ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))  # Major ticks every 1000
ax.xaxis.set_minor_locator(ticker.MultipleLocator(500))   # Minor ticks every 500
plt.show()


**Average and replicate along horizontal x-axis**

In [ ]:
v = np.repeat(np.mean(v, axis=1)[:, np.newaxis], np.max(orig_shape), axis=1)
z = np.repeat(np.mean(z, axis=1)[:, np.newaxis], np.max(orig_shape), axis=1)

fig, ax = plt.subplots(figsize=(12, 4))
ax.imshow(v)
ax.set_aspect('auto')
ax.set_yticks([])
ax.minorticks_on()
ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))  # Major ticks every 1000
ax.xaxis.set_minor_locator(ticker.MultipleLocator(500))   # Minor ticks every 500
plt.show()


if orig_shape[0] > orig_shape[1]:
    v = np.swapaxes(v, 0, 1)
    z = np.swapaxes(z, 0, 1)

print(f"Output flat-field model shape: {v.shape}")

save_dir = outdir / f'{wl}_{sample}_S{slc}'
if not save_dir.exists():
    save_dir.mkdir(parents=True, exist_ok=True)
    
tiff.imwrite(save_dir / 'v.tif', v, compression='zlib')
tiff.imwrite(save_dir / 'z.tif', z, compression='zlib')
